In [5]:
# Install newer version of Pytorch
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

# Cloning ONLY colab-integration branch
from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Github password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone -b colab-integration --single-branch "https://$GITHUB_AUTH@github.com/baltekgajda/LegoSortingRecognition.git"

# Navigate to project folder
os.chdir("./LegoSortingRecognition")

Github user··········
Github password··········
Cloning into 'LegoSortingRecognition'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 186 (delta 99), reused 124 (delta 49), pack-reused 0
Receiving objects: 100% (186/186), 63.89 KiB | 703.00 KiB/s, done.
Resolving deltas: 100% (99/99), done.


In [0]:
# Import necessary libraries
from __future__ import print_function
from __future__ import division
from google.colab import drive
import numpy as np
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import json
import time

from data_loader import load_data
from feature_extraction import train_classifier_only, train_classifier_and_last_conv, train_full_net, train_simplified_net
from net_test_and_metrics import test_network
from svm_classification import train_model_with_svm
import VGGFactory

In [7]:
# Mount Google Drive to access data
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def save_to_json(variable, results_folder, file_name):
  as_json = json.dumps(variable)
  f = open(RESULTS_FOLDER + file_name,"w")
  f.write(as_json)
  f.close()

In [0]:
def save_to_csv(variable, results_folder, file_name):
    np.savetxt(results_folder + file_name, variable, delimiter=',')

In [0]:
def create_folders():
  t = str(time.time())
  root = '/content/drive/My Drive/Studia/SNR'
  current_execution_dir = '/content/drive/My Drive/Studia/SNR/{}'.format(t)
  models_dir = '{}/models/'.format(current_execution_dir)
  results_dir = '{}/results/'.format(current_execution_dir)
  os.chdir(root)
  os.mkdir(current_execution_dir)
  os.mkdir(models_dir)
  os.mkdir(results_dir)
  
  return (models_dir, results_dir)

In [0]:
models_dir, results_dir = create_folders()
NUM_OF_CLASSES = 20
NUM_OF_EPOCHS = 50
INPUT_SIZE = 224
DATA_DIR = '/content/drive/My Drive/Studia/SNR/lego/'
MODELS_FOLDER = models_dir
RESULTS_FOLDER = results_dir

print("Initializing Datasets and Dataloaders...")
dataloaders_dict = load_data(DATA_DIR, INPUT_SIZE, batch_size=4, num_workers=4)


if (torch.cuda.is_available() == False):
  raise RuntimeError("GPU is not available!")   
device = torch.device("cuda:0")

torch.cuda.current_device()


model, hist_classifier, test_hist  = train_full_net(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
metrics = test_network(model, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc')
save_to_json(test_hist, RESULTS_FOLDER, "test_hist_on_full_net.json")
save_to_json(hist_classifier, RESULTS_FOLDER, "hist_on_full_net.json")
save_to_json(metrics, RESULTS_FOLDER, "metrics_on_full_net.json")

m_simplified, hist_classifier, test_hist  = train_simplified_net(model, dataloaders_dict, MODELS_FOLDER, device, num_of_epochs=NUM_OF_EPOCHS)
metrics = test_network(m_simplified, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc')
save_to_json(test_hist, RESULTS_FOLDER, "test_hist_simplified.json")
save_to_json(hist_classifier, RESULTS_FOLDER, "hist_simplified.json")
save_to_json(metrics, RESULTS_FOLDER, "metrics_simplified.json")

model, hist_classifier, test_hist = train_classifier_only(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
metrics = test_network(model, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc')
save_to_json(test_hist, RESULTS_FOLDER, "test_hist_classifier.json")
save_to_json(hist_classifier, RESULTS_FOLDER, "hist_classifier.json")
save_to_json(metrics, RESULTS_FOLDER, "metrics_classifier.json")

model, hist_classifier, test_hist  = train_classifier_and_last_conv(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
metrics = test_network(model, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc')
save_to_json(test_hist, RESULTS_FOLDER, "test_hist_hist_last_conv.json")
save_to_json(hist_classifier, RESULTS_FOLDER, "hist_last_conv.json")
save_to_json(metrics, RESULTS_FOLDER, "metrics_last_conv.json")

svm_model = train_model_with_svm(m_simplified, dataloaders_dict, kernel='l')
metrics = test_network(m_simplified, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc', svm_classifier=svm_model)
save_to_json(metrics, RESULTS_FOLDER, "metrics_linear_svm.json")

svm_model = train_model_with_svm(m_simplified, dataloaders_dict, kernel='q')
metrics = test_network(m_simplified, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc', svm_classifier=svm_model)
save_to_json(metrics, RESULTS_FOLDER, "metrics_quadratic_svm.json")

svm_model  = train_model_with_svm(m_simplified, dataloaders_dict, kernel='e')
metrics = test_network(m_simplified, dataloaders_dict['test'], NUM_OF_CLASSES, device, plot_name='cmc', svm_classifier=svm_model)
save_to_json(metrics, RESULTS_FOLDER, "metrics_exponential_svm.json")

Initializing Datasets and Dataloaders...
Epoch 0/49
----------
train Loss: 2.9515 Acc: 0.1298
val Loss: 2.7913 Acc: 0.2900
Epoch 1/49
----------
train Loss: 2.6880 Acc: 0.3944
val Loss: 2.5784 Acc: 0.5054
Epoch 2/49
----------
train Loss: 2.5505 Acc: 0.5286
val Loss: 2.4801 Acc: 0.6006
Epoch 3/49
----------
train Loss: 2.4895 Acc: 0.5883
val Loss: 2.4727 Acc: 0.6061
Epoch 4/49
----------
train Loss: 2.4709 Acc: 0.6087
val Loss: 2.4595 Acc: 0.6190
Epoch 5/49
----------
train Loss: 2.4563 Acc: 0.6216
val Loss: 2.4779 Acc: 0.5952
Epoch 6/49
----------
train Loss: 2.4554 Acc: 0.6216
val Loss: 2.4862 Acc: 0.5909
Epoch 7/49
----------
train Loss: 2.4308 Acc: 0.6457
val Loss: 2.4308 Acc: 0.6472
Epoch 8/49
----------
train Loss: 2.3994 Acc: 0.6801
val Loss: 2.3978 Acc: 0.6818
Epoch 9/49
----------
train Loss: 2.3933 Acc: 0.6853
val Loss: 2.4378 Acc: 0.6407
Epoch 10/49
----------
train Loss: 2.3680 Acc: 0.7104
val Loss: 2.3948 Acc: 0.6818
Epoch 11/49
----------
train Loss: 2.3511 Acc: 0.7286
va

In [0]:
# This probably can be deleted

# m_classifier, hist_classifier = train_classifier_only(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
# metrics_classifier = test_network(m_classifier, dataloaders_dict['test'], device, plot_name='cmc')
# save_to_json(hist_classifier, RESULTS_FOLDER, "hist_classifier.json")
# save_to_json(metrics_classifier, RESULTS_FOLDER, "metrics_classifier.json")

# m_last_conv, hist_last_conv = train_classifier_and_last_conv(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
# metrics_last_conv = test_network(m_last_conv, dataloaders_dict['test'], device, plot_name='cmc')
# save_to_json(hist_last_conv, RESULTS_FOLDER, "hist_last_conv.json")
# save_to_json(metrics_last_conv, RESULTS_FOLDER, "metrics_last_conv.json")

# m_on_full_net, hist_on_full_net = train_full_net(dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
# metrics_on_full_net = test_network(m_on_full_net, dataloaders_dict['test'], device, plot_name='cmc')
# save_to_json(hist_on_full_net, RESULTS_FOLDER, "hist_on_full_net.json")
# save_to_json(metrics_on_full_net, RESULTS_FOLDER, "metrics_on_full_net.json")

# m_simplified, hist_simplified = train_simplified_net(model_on_full_net, dataloaders_dict, MODELS_FOLDER, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
# metrics_simplified_net = test_network(m_simplified, dataloaders_dict['test'], device, plot_name='cmc')
# save_to_json(hist_simplified, RESULTS_FOLDER, "hist_simplified.json")
# save_to_json(metrics_on_full_net, RESULTS_FOLDER, "metrics_on_full_net.json")

In [0]:
import os
import shutil

os.chdir("/content")
shutil.rmtree('./LegoSortingRecognition')